In [7]:
# 1️⃣  Mount Drive
from google.colab import drive
drive.mount('/content/drive')        # follow the link → copy auth code

# 2️⃣  Verify the file is really there
!ls -lh "/content/drive/MyDrive/raw.csv"

# 3️⃣  Load the data (index is the first column)
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/raw.csv",
                 index_col=0,            # first column is the date index
                 parse_dates=True)

print(df.shape)
df.tail()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
-rw------- 1 root root 265K Jun 29 22:09 /content/drive/MyDrive/raw.csv
(3854, 4)


,GC=F,DX-Y.NYB,CL=F,CPI
2025-04-25,3282.399902,99.470001,63.020000,320.321
2025-04-28,3332.500000,99.010002,62.049999,320.321
2025-04-29,3318.800049,99.239998,60.419998,320.321
2025-04-30,3305.000000,99.470001,58.209999,320.321
2025-05-01,3210.000000,100.250000,59.240002,320.580


In [8]:
!pip -q install ta

In [9]:
import pandas as pd, numpy as np, pickle
from sklearn.preprocessing import MinMaxScaler
# !pip -q install ta          # tech-indicator helper
import ta

In [10]:
# 1. Load raw file from Drive (mounted earlier)
df = pd.read_csv("/content/drive/MyDrive/raw.csv",
                 index_col=0, parse_dates=True)

# 2. Create a working copy for new features
feat = df.copy()

# ────── Core tickers (leave names untouched) ──────
gold = "GC=F"
usd  = "DX-Y.NYB"
oil  = "CL=F"

# 3. Percentage returns
feat["Gold_ret"] = feat[gold].pct_change()
feat["USD_ret"]  = feat[usd ].pct_change()
feat["Oil_ret"]  = feat[oil ].pct_change()

# 4. Moving averages and volatility on gold
feat["MA5"]   = feat[gold].rolling(5).mean()
feat["MA20"]  = feat[gold].rolling(20).mean()
feat["Vol10"] = feat[gold].rolling(10).std()

# 5. RSI (14-day) on gold
feat["RSI14"] = ta.momentum.RSIIndicator(close=feat[gold], window=14).rsi()

# 6. Drop rows with NaNs from rolling windows
feat.dropna(inplace=True)
print("Feature matrix shape:", feat.shape)

# 7. Scale 0-1
scaler = MinMaxScaler()
feat_scaled = pd.DataFrame(
    scaler.fit_transform(feat),
    index  = feat.index,
    columns= feat.columns)

# 8. Build 30-day windows → predict next-day gold price
import numpy as np, pickle
from sklearn.preprocessing import MinMaxScaler

LOOKBACK = 30
Xs, ys = [], []

for i in range(len(feat_scaled) - LOOKBACK):
    Xs.append(feat_scaled.iloc[i:i+LOOKBACK].values)
    ys.append(feat_scaled.iloc[i+LOOKBACK][gold])   # next-day gold (scaled)

Xs, ys = np.array(Xs), np.array(ys)

# ─── Split: last 760 samples = validation ───
VAL_LEN  = 760                           # ≈ 20 %
X_train  = Xs[:-VAL_LEN]
y_train  = ys[:-VAL_LEN]
X_val    = Xs[-VAL_LEN:]
y_val    = ys[-VAL_LEN:]

# ─── Persist to Drive ───
DRIVE = "/content/drive/MyDrive"
np.save(f"{DRIVE}/train.npy",   X_train)
np.save(f"{DRIVE}/y_train.npy", y_train)   # NEW file
np.save(f"{DRIVE}/val.npy",     X_val)
np.save(f"{DRIVE}/y_val.npy",   y_val)
pickle.dump(scaler, open(f"{DRIVE}/scaler.pkl", "wb"))

print("Saved:")
print("  train.npy  ", X_train.shape)
print("  y_train.npy", y_train.shape)
print("  val.npy    ", X_val.shape)
print("  y_val.npy  ", y_val.shape)

Feature matrix shape: (3835, 11)
Saved:
  train.npy   (3045, 30, 11)
  y_train.npy (3045,)
  val.npy     (760, 30, 11)
  y_val.npy   (760,)
